# Endpoint en Tiempo Real con Lambda.

- Para probar los endpoits podemos usar boto3 o AWS CLI.

```bash
aws sagemaker-runtime invoke-endpoint \
    --endpoint-name endpoint_name \
    --body fileb://$file_name \
    output_file.txt
```
- También desde la interfaz de sagemaker studio.
- Doc: https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/realtime-endpoints-test-endpoints.html

#### Datos de prueba

In [28]:
import pandas as pd
df_test = pd.read_csv('test.csv', header=None)

In [33]:
to_try = df_test.iloc[[1,2, 497, 499], :]
to_try

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1,0,126,0,0,0,3.816301,2,7.070955,5.420410,9,3.101070,8.989511,250,0.424210,4.142589,5,10.617313,1
2,0,52,1,0,0,4.761451,2,6.575281,3.535719,6,3.770251,5.043752,200,4.594888,6.052541,5,3.918532,4
497,1,170,1,1,100,11.835835,3,9.530642,6.967019,4,7.016593,3.317470,200,7.574951,5.070610,7,3.012213,5
499,1,65,0,1,200,6.171609,2,8.821492,6.816145,6,4.641622,4.568890,250,3.982190,4.586354,3,6.766862,5


#### Llamada al endpooint en tiempo real

In [37]:
csv_string = to_try.iloc[:, 1:].to_csv(index=False, header=False)
csv_string

'126,0,0,0,3.816301464693282,2,7.0709549060195,5.420409753405697,9,3.1010696967511544,8.989510760940297,250,0.4242104752880697,4.142588527377276,5,10.617313362277589,1\n52,1,0,0,4.761451007673512,2,6.5752810298200295,3.5357186950716923,6,3.7702507576575233,5.043751831924423,200,4.594888172955266,6.052541264501843,5,3.918532222373023,4\n170,1,1,100,11.83583513990359,3,9.530641950015236,6.967019023053207,4,7.016592819269271,3.31747015427766,200,7.574950710886639,5.0706097607076615,7,3.012212995399225,5\n65,0,1,200,6.171608597774607,2,8.821491509134809,6.816145096011792,6,4.641622257072825,4.568889905414934,250,3.982190140834256,4.586353731276165,3,6.7668617408900165,5\n'

In [38]:
import boto3

# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account. 
endpoint_name='chun-xgboost-severless'

# After you deploy a model into production using SageMaker hosting 
# services, your client applications use this API to get inferences 
# from the model hosted at the specified endpoint.
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=csv_string,
)

# Optional - Print the response body and decode it so it is human read-able.
print(response['Body'].read().decode('utf-8'))

0.1629495471715927
0.3985714912414551
0.9913638830184937
0.3341834247112274



### Invocación desde una función lambda
- https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/

In [ ]:
import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = 'chun-xgboost-severless'
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    
    data = json.loads(json.dumps(event))
    payload = data['data']
    print(payload)
    
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/csv',
                                       Body=payload)
    print(response)
    result = json.loads(response['Body'].read().decode())
    print(result)
    pred = int(result['predictions'][0]['score'])
    predicted_label = 'M' if pred == 1 else 'B'
    
    return predicted_label